# Access and visualization of EO Data on the Cloud

**Purpose** : 
>Connect to an EO data catalog by using a [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/) client in Python.\
>Search for data products for a specific region and time period.\
>Display the boundaries of the available scenes\
>Get an overview of the actual images\
>Display a high-resolution natural color composite image for a small region in the study area.

<hr/> 

## 1-import required libraries

In [ ]:
# Import necessary packages
import os 
import cv2
from datetime import date
import json
import itertools
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import pixel_to_utm, utm_to_pixel
import utm
import pandas as pd
from shapely.geometry import box
from rasterio.mask import mask
from pystac_client import Client
import shapely.geometry
import shapely.wkt
import folium
import shapely.geometry
from odc.algo import to_rgba

<hr/> 

## 2-Set the AOI

[Choose aoi](https://geojson.io/#map=3.07/8.1/21.2)

In [ ]:
#Set the path to data
aoi_dir = "./aoi"
aoi_file = "mark.geojson"
aoi_path = os.path.join(aoi_dir , aoi_file ) 

In [ ]:
def get_bounds_of_AoI(obj_aoi, offset):
    
    aoi = gpd.read_file(obj_aoi)
    
    bounds = aoi.total_bounds
    #offset = 1/60  #200m in degree
    # Extend the bounding box by 200 m
    minx, miny = bounds[0]-offset, bounds[1]-offset
    maxx, maxy = bounds[2]+offset, bounds[3]+offset

    bbox = box(minx, miny, maxx, maxy)
    
    print(bbox)

    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs="EPSG:4326")

    
    return geo 

In [ ]:
#offset = 1/60  #200m in degree
bbx = get_bounds_of_AoI(aoi_path, 1/60)

In [ ]:
bbx.explore()

### 3-Connect to the catalog and query available products

In [ ]:
values = []
for i in range(len(bbx.total_bounds)):
    values.append(bbx.total_bounds[i])
keys = ["lonmin", "latmin", "lonmax", "latmax"]  
geometry = dict(zip(keys, values))

In [ ]:
geometry

In [ ]:
# Import Python library for working with STAC (https://github.com/stac-utils/pystac-client)
bbox = [ geometry["lonmin"], geometry["latmin"],  geometry["lonmax"], geometry["latmax"]]
# Set time period
time = ["2023-03-01", "2023-03-31"]
#time = ["2021-01-01", "2023-03-15"]
# Connect to the AWS Earth Search catalog
catalog = Client.open("https://earth-search.aws.element84.com/v0")

# Query datasets
query = catalog.search(
    collections=["sentinel-s2-l2a-cogs"],
    datetime=time,
    limit=100,
    bbox=bbox,
    query = {  
      "eo:cloud_cover":{  
        "lt":"10"
          #"lt":"1"
      }
    }
)

# Display number of items found
items = list(query.get_items())
print(f"Found: {len(items):d} items")

In [ ]:
# Display properties of the first item
items[0].properties

## 4-Display granules of the available items

In [ ]:
%%capture --no-display
# Create a geo-dataframe from the datasets (WGS84)
gdf = gpd.GeoDataFrame.from_features(query.get_all_items_as_dict(), "epsg:4326")

# Compute granule ids
gdf["granule"] = (
    gdf["sentinel:utm_zone"].apply(lambda x: f"{x:02d}")
    + gdf["sentinel:latitude_band"]
    + gdf["sentinel:grid_square"]
)

# Create map
map1 = folium.Map()

# Add study area to the map
folium.GeoJson(
    shapely.geometry.box(*bbox),
    style_function=lambda x: dict(fill=False, weight=3, opacity=0.7, color="red"),
).add_to(map1)

# Add dataset granules to the map
gdf.explore(
    "granule",
    categorical=True,
    tooltip=[
        "granule",
        "datetime",
        "sentinel:data_coverage",
        "eo:cloud_cover",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    m=map1,
)

# Zoom to the granules
bounds = gdf.unary_union.bounds
map1.fit_bounds(bounds=((bounds[1], bounds[0]), (bounds[3], bounds[2])))

# Display map
map1

## 5-Load data

In [ ]:
# Import tools for converting STAC metadata to ODC data model
from odc.stac import configure_rio, stac_load

# Load* data (WGS 84 Pseudo-Mercator)
data = stac_load(
    items,
    bands=("B04", "B03", "B02"),
    crs="epsg:3857",
    resolution=10 * 2**5,
    #resolution=10, 
)
data

## 6-Create natural-color composites

In [ ]:
%%time
# Load miscellaneous algorithmic helper methods
# Generate* natural-color composite image
rgba = to_rgba(data, clamp=(1, 3000), bands=("B04", "B03", "B02")).compute()

## 7-Display natural-color composites

In [ ]:
# Create map
map2 = folium.Map()

# Add study area to the map
folium.GeoJson(
    shapely.geometry.box(*bbox),
    style_function=lambda x: dict(fill=False, weight=3, opacity=0.7, color="red"),
    name="Study Area",
).add_to(map2)

# Add dataset granules to the map
gdf.explore(
    "granule",
    categorical=True,
    tooltip=False,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="Granules",
    m=map2,
)

# Get geographic extent of the images
extent = rgba.geobox.geographic_extent.boundingbox
bounds = ((extent[3], extent[0]), (extent[1], extent[2]))

# Add images 
for i in range(rgba.coords["time"].size):
    folium.raster_layers.ImageOverlay(
        rgba.isel(time=i).data, bounds=bounds, name="Image {}".format(i + 1)
    ).add_to(map2)

# Add layer control
folium.LayerControl().add_to(map2)

# Zoom to the images
map2.fit_bounds(bounds=bounds)

# Display map
map2

## Load high-resolution data

In [ ]:
# Set smaller study area
#offset = 1/60  #200m in degree
aoi = get_bounds_of_AoI(aoi_path, 0)

In [ ]:
values = []
for i in range(len(bbx.total_bounds)):
    values.append(bbx.total_bounds[i])
keys = ["lonmin", "latmin", "lonmax", "latmax"]  
aoi_geom = dict(zip(keys, values))

In [ ]:
small_bbox = [ aoi_geom["lonmin"], aoi_geom["latmin"],  aoi_geom["lonmax"], aoi_geom["latmax"]]

In [ ]:
small_bbox =small_bbox

# Load data
small_data = stac_load(
    items,
    bands=("B04", "B03", "B02"),
    crs="epsg:3857",
    resolution=10,
    chunks={},
    groupby="solar_day",
    bbox=small_bbox,
)

In [ ]:
%%time
small_rgba = to_rgba(small_data, clamp=(1, 3000), bands = ("B04", "B03", "B02")).compute()
small_rgba.coords

## Display high-resolution data

In [ ]:
# Import notebook display helper methods
import odc.ui
from IPython.display import HTML

# Generate JPEG image of the first image
img = odc.ui.mk_data_uri(odc.ui.to_jpeg_data(small_rgba.isel(time=0).data))

# Display image
HTML(data=f"""<img src="{img}" width='100%'>""")

In [ ]:
# Display first two images side by side
img_a = odc.ui.mk_data_uri(odc.ui.to_jpeg_data(small_rgba.isel(time=0).data))
img_b = odc.ui.mk_data_uri(odc.ui.to_jpeg_data(small_rgba.isel(time=1).data))

HTML(data=f"""<img src="{img_a}" style="width:50%; float: left"><img src="{img_b}" style="width:50%; float: left">""")

## Save as GeoTIFF

In [ ]:
# Import rasterio xarray extension
import rioxarray

# Save low-resolution images
for i in range(rgba.coords["time"].size):
    rgba.isel(time=i).transpose('band', 'y', 'x').rio.to_raster("RGB_{}.tif".format(i))

# Save high-resolution images
for i in range(small_rgba.coords["time"].size):
    small_rgba.isel(time=i).transpose('band', 'y', 'x').rio.to_raster("RGB_Small_{}.tif".format(i))